In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -U -r requirements.txt   # install

Cloning into 'yolov5'...
remote: Enumerating objects: 16008, done.
remote: Total 16008 (delta 0), reused 0 (delta 0), pack-reused 16008
Receiving objects: 100% (16008/16008), 14.56 MiB | 25.11 MiB/s, done.
Resolving deltas: 100% (11011/11011), done.
/content/yolov5/yolov5
ERROR: Operation cancelled by user


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd ..
%ls

In [ ]:
%ls

In [ ]:
%mkdir datasets
%mkdir datasets/wood_defects_1
%mkdir datasets/wood_defects_1/images
%mkdir datasets/wood_defects_1/images/test
%mkdir datasets/wood_defects_1/images/train

%mkdir datasets/wood_defects_1/labels
%mkdir datasets/wood_defects_1/labels/test
%mkdir datasets/wood_defects_1/labels/train

In [ ]:
%ls

In [ ]:
# importing the data from kaggle
! pip install kaggle
! mkdir ~/.kaggle
! cp ./drive/MyDrive/yolov5/kaggle.json ~/.kaggle/



In [ ]:
! kaggle datasets download pjames266/brents-wood-1
! kaggle datasets download pjames266/brents-wood-2
#! kaggle datasets download pjames266/brents-wood-3
#! kaggle datasets download pjames266/brents-wood-4

In [ ]:
%ls

In [ ]:
!unzip ./brents-wood-1.zip -d ./datasets/wood_defects_1/images
!unzip ./brents-wood-2.zip -d ./datasets/wood_defects_1/images
#!unzip ./brents-wood-3.zip -d ./datasets/wood_defects_1/images
#!unzip ./brents-wood-4.zip -d ./datasets/wood_defects_1/images

In [ ]:
! unzip ./drive/MyDrive/yolov5/Bouding_Boxes.zip -d ./datasets/wood_defects_1/labels

In [ ]:
import glob
import random
import os
import re


%ls

img_path = './datasets/wood_defects_1/images/'
img_files = glob.iglob(img_path + '*.bmp')
label_path = r'datasets/wood_defects_1/labels/Bouding Boxes/'
label_files = glob.iglob(label_path + '*.txt')

#remove _anno
for i, label in enumerate(label_files):
  os.rename(label, re.sub("_anno", '', label))

In [ ]:
#delete all non-existing img labels
label_files = glob.iglob(label_path + '*.txt')

for q,label in enumerate(label_files):
  img_name = os.path.basename(label).split('.')[0]

  if not os.path.exists(img_path + img_name + '.bmp'):
    os.remove(label)

In [ ]:
#refactor lines of all labels, where each row is class x_center y_center width height
label_files = glob.iglob(label_path + '*.txt')

class_index = {"Live_Knot": "0", "Dead_Knot": "1", "knot_with_crack": "2", "resin": "3", "Knot_missing": "4", "Crack": "5",
               "Quartzity": "6", "Marrow": "7"}
label_files = glob.iglob(label_path + '*.txt')
for label in label_files:
  img_name = os.path.basename(label).split('.')[0]
  with open(label, 'r') as f:
    lines = f.readlines()
    for line in lines:
      items = line.split("\t")

      items[0] = class_index[items[0]]
      items[1] = re.sub(',', '.', items[1])
      items[2] = re.sub(',', '.', items[2])
      items[3] = re.sub(',', '.', items[3])
      items[4] = re.sub(',', '.', items[4])
      left,top,bottom,right = float(items[1]),float(items[2]),float(items[3]),float(items[4][:-2])
      width = abs(right - left)
      height = abs(top - bottom)
      x_center = (left + right)/2
      y_center = (top + bottom)/2
      new_line = "\t".join([items[0], str(x_center), str(y_center), str(width), str(height) + "\n"])
  with open(label, 'w') as f:
      f.writelines(new_line)


In [ ]:
#split the image/txt into train/test dataset
import math
import shutil

label_files = glob.iglob(label_path + '*.txt')
print(label_path)
#label_files = random.shuffle(label_files)
length = sum(1 for _ in label_files)
print(length)

training_n = math.floor(0.8*length) #80% training
print(training_n)

test_n = length - training_n

label_files = glob.iglob(label_path + '*.txt')

#loop through all labels
for i,label in enumerate(label_files):

  fname = os.path.basename(label)
  img_name = fname.split('.')[0] + ".bmp"

  label_training_path = 'datasets/wood_defects_1/labels/train/' + fname
  label_test_path = 'datasets/wood_defects_1/labels/test/' + fname

  img_path = 'datasets/wood_defects_1/images/' + img_name
  img_training_path = 'datasets/wood_defects_1/images/train/' + img_name
  img_test_path = 'datasets/wood_defects_1/images/test/' + img_name

  #for training data
  if i <= training_n:
    #print(label)
    #print(label_training_path)
    shutil.move(label, label_training_path)

    #print(img_path)
    #print(img_training_path)
    shutil.move(img_path,img_training_path)
  else:
    shutil.move(label,label_test_path)
    shutil.move(img_path, img_test_path)


In [ ]:
%cp ./drive/MyDrive/yolov5/wood_defects_1.yaml ./yolov5/data/
%cp ./drive/MyDrive/yolov5/yolov5s.pt ./yolov5/ #not automatically downloading weights

In [ ]:
%cd ./yolov5

In [ ]:
%%python train.py --img 1280 --epochs 50 --data wood_defects_1.yaml --weights yolov5s.pt